# Prediction from classifier

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
cudnn.benchmark = True
plt.ion()   # interactive mode
from utils.TestImagesDataset import TestImagesDataset
from tqdm import tqdm
CLASSES=14
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu")

In [2]:
data_transforms = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_set = TestImagesDataset(data_transforms)
test_set.__len__()

184003

In [3]:
bs=256
dataset_size = test_set.__len__()
dataloader = DataLoader(dataset=test_set,
                          batch_size=bs,
                          shuffle=False,
                          num_workers=4)

In [5]:
EFFICIENTNET=True
model_ft=None

if EFFICIENTNET:
    model_ft = models.efficientnet_b0(pretrained=True)
    model_ft.classifier[1] = nn.Linear(in_features=1280, out_features=CLASSES)
    model_ft = model_ft.to(device)
    #reloading saved weights
    model_ft.load_state_dict(torch.load('./models/efficientnet_b0_noval.pt'))
    model_ft.eval()
else: # RESNET
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, CLASSES)
    model_ft = model_ft.to(device)

    #reloading saved weights
    model_ft.load_state_dict(torch.load('./models/resnet18.pt'))
    model_ft.eval()



with torch.no_grad():
    n_correct = 0
    n_samples = 0
    predictions=[]
    with tqdm(dataloader, unit="batch") as tepoch: 
        for images in tepoch:
            tepoch.set_description("Predicting...")
            images = images.to(device)
            outputs = model_ft(images)
            _, preds = torch.max(outputs, 1)
           
            predictions.append(preds)


Predicting...: 100%|██████████████████████████████████████████████████████████| 719/719 [05:23<00:00,  2.23batch/s]


In [6]:
all_preds=torch.cat(predictions, dim=0).cpu().detach().numpy()

In [7]:
paths=[]
for i in range(71,126):
    paths.append('RALIHR_surgeon01_fps01_'+('0000'+str(i))[-4:])

for i in range(1,5):
    paths.append('RALIHR_surgeon02_fps01_'+('0000'+str(i))[-4:])

paths.append('RALIHR_surgeon03_fps01_0001')
vid2len={}
import os
ROOT='./e6691-bucket-images/'
index2data=[]
for vid in paths:
    vid2len[vid]=len(os.listdir(os.path.join(ROOT,vid)))
    for i in range (len(os.listdir(os.path.join(ROOT,vid)))):
        index2data.append(os.path.join(vid,'frame'+str(i)+'.jpg'))
        

In [8]:
import pandas as pd
df=pd.read_csv('./kaggle_template.csv')

In [25]:
vid2len

{'RALIHR_surgeon01_fps01_0071': 1321,
 'RALIHR_surgeon01_fps01_0072': 4333,
 'RALIHR_surgeon01_fps01_0073': 1918,
 'RALIHR_surgeon01_fps01_0074': 4173,
 'RALIHR_surgeon01_fps01_0075': 2479,
 'RALIHR_surgeon01_fps01_0076': 2467,
 'RALIHR_surgeon01_fps01_0077': 1530,
 'RALIHR_surgeon01_fps01_0078': 2172,
 'RALIHR_surgeon01_fps01_0079': 3478,
 'RALIHR_surgeon01_fps01_0080': 3425,
 'RALIHR_surgeon01_fps01_0081': 1913,
 'RALIHR_surgeon01_fps01_0082': 5125,
 'RALIHR_surgeon01_fps01_0083': 3060,
 'RALIHR_surgeon01_fps01_0084': 1379,
 'RALIHR_surgeon01_fps01_0085': 5593,
 'RALIHR_surgeon01_fps01_0086': 3400,
 'RALIHR_surgeon01_fps01_0087': 1907,
 'RALIHR_surgeon01_fps01_0088': 2341,
 'RALIHR_surgeon01_fps01_0089': 5706,
 'RALIHR_surgeon01_fps01_0090': 2542,
 'RALIHR_surgeon01_fps01_0091': 4413,
 'RALIHR_surgeon01_fps01_0092': 2833,
 'RALIHR_surgeon01_fps01_0093': 3865,
 'RALIHR_surgeon01_fps01_0094': 1478,
 'RALIHR_surgeon01_fps01_0095': 2239,
 'RALIHR_surgeon01_fps01_0096': 2103,
 'RALIHR_sur

In [26]:
df

,Id,Predicted
0,001-0071-00001,transitionary idle
1,001-0071-00002,transitionary idle
2,001-0071-00003,transitionary idle
3,001-0071-00004,transitionary idle
4,001-0071-00005,transitionary idle
...,...,...
179863,003-0001-05902,transitionary idle
179864,003-0001-05903,transitionary idle
179865,003-0001-05904,transitionary idle
179866,003-0001-05905,transitionary idle


In [35]:
all_preds.shape

(184003,)

In [60]:
predictions=all_preds
correct_length_seq=[]
for vid in vid2len:
    seq=predictions[:vid2len[vid]].tolist()
    predictions=predictions[vid2len[vid]:]
    rows=vid[14:16]+'-'+vid[23:27]+'-'
    subdf=df[df['Id'].str.contains(rows)]
    indexes=[]
    for index, row in subdf.iterrows():
        frame=int(row['Id'][9:])
        indexes.append(frame-1)
    correct_length_seq.append([seq[index] for index in indexes])
    print(vid)
len(correct_length_seq)

RALIHR_surgeon01_fps01_0071
RALIHR_surgeon01_fps01_0072
RALIHR_surgeon01_fps01_0073
RALIHR_surgeon01_fps01_0074
RALIHR_surgeon01_fps01_0075
RALIHR_surgeon01_fps01_0076
RALIHR_surgeon01_fps01_0077
RALIHR_surgeon01_fps01_0078
RALIHR_surgeon01_fps01_0079
RALIHR_surgeon01_fps01_0080
RALIHR_surgeon01_fps01_0081
RALIHR_surgeon01_fps01_0082
RALIHR_surgeon01_fps01_0083
RALIHR_surgeon01_fps01_0084
RALIHR_surgeon01_fps01_0085
RALIHR_surgeon01_fps01_0086
RALIHR_surgeon01_fps01_0087
RALIHR_surgeon01_fps01_0088
RALIHR_surgeon01_fps01_0089
RALIHR_surgeon01_fps01_0090
RALIHR_surgeon01_fps01_0091
RALIHR_surgeon01_fps01_0092
RALIHR_surgeon01_fps01_0093
RALIHR_surgeon01_fps01_0094
RALIHR_surgeon01_fps01_0095
RALIHR_surgeon01_fps01_0096
RALIHR_surgeon01_fps01_0097
RALIHR_surgeon01_fps01_0098
RALIHR_surgeon01_fps01_0099
RALIHR_surgeon01_fps01_0100
RALIHR_surgeon01_fps01_0101
RALIHR_surgeon01_fps01_0102
RALIHR_surgeon01_fps01_0103
RALIHR_surgeon01_fps01_0104
RALIHR_surgeon01_fps01_0105
RALIHR_surgeon01_fps

60

### cleaning sequences

In [61]:
import torch.nn.functional as F
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()        
        self.lstm = nn.LSTM(input_size=14, hidden_size=32, num_layers=2,bidirectional=True)
        #self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(64, CLASSES)
       
    def forward(self, x_3d):
        #x3d:  torch.Size([128, 32, 1, 80, 80])
        hidden = None
        toret  = []
        
        output, hidden = self.lstm(x_3d, hidden)
       

        x = F.relu(output)
        x = self.fc2(x)
        #print("x shape: ", x.shape)
            
            
        return x
model = LSTM()
model.to(device)
model.load_state_dict(torch.load('./models/lstm.pt'))
model.eval()


LSTM(
  (lstm): LSTM(14, 32, num_layers=2, bidirectional=True)
  (fc2): Linear(in_features=64, out_features=14, bias=True)
)

In [62]:
def clean(seq):
    #onehot encode
    seq=np.asarray(seq)
    onehot = np.zeros((seq.size, 14)) 
    onehot[np.arange(seq.size),seq] = 1
    #onehot=np.expand_dims(onehot, axis=0)
    #print(onehot.shape)
    #1,len,14
    x=torch.Tensor.float(torch.from_numpy(onehot))
    with torch.no_grad():
        x = x.to(device)
        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        
        return preds.cpu().detach().numpy().tolist()


In [63]:
clean_seq=[]
for seq in correct_length_seq:
    cleaned=clean(seq)
    clean_seq.append(cleaned)


In [64]:
correct_length_seq

[[13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  9,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  9,
  7,
  7,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  7,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,

In [65]:
clean_seq

[[13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  9,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  9,
  7,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,

In [66]:
df['Predicted'] =[item for sublist in clean_seq for item in sublist]

In [67]:
num2labels={0:"adhesiolysis",
           1:'blurry',
           2:"catheter insertion",
           3:'mesh placement',
           4:'mesh positioning',
           5:"out of body",
           6:"peritoneal closure",
           7:"peritoneal scoring",
           8:'positioning suture',
           9:'preperioneal dissection',
           10:"primary hernia repair",
           11:"reduction of hernia",
           12:"stationary idle",
           13:"transitionary idle"}

In [68]:
df['Predicted'] = df['Predicted'].map(num2labels)

In [23]:
df

,Id,Predicted
0,001-0071-00001,transitionary idle
1,001-0071-00002,transitionary idle
2,001-0071-00003,transitionary idle
3,001-0071-00004,transitionary idle
4,001-0071-00005,transitionary idle
...,...,...
179863,003-0001-05902,transitionary idle
179864,003-0001-05903,transitionary idle
179865,003-0001-05904,transitionary idle
179866,003-0001-05905,transitionary idle


In [69]:
df.to_csv('./my_kaggle_preds.csv',index=False)